In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns, warnings

%matplotlib inline
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family='NanumGothic')
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, SVR
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from xgboost import XGBRegressor

In [76]:
df_raw_1 = pd.read_csv('./data/jeju_data_ver1/201901-202003.csv')
df_raw_2 = pd.read_csv('./data/jeju_data_ver1/202004.csv')

df_raw = pd.concat([df_raw_1, df_raw_2])

In [146]:
df_raw.tail()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
1350317,202004,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5
1350318,202004,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6
1350319,202004,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5
1350320,202004,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7
1350321,202004,충북,충주시,휴양콘도 운영업,충북,충주시,60s,2,5,3,227000,4


# Data Preprocessing

In [185]:
df = df_raw.copy()

In [186]:
df.fillna('세종시', inplace=True)

In [187]:
df.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [188]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [189]:
df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

In [190]:
df.drop(['CARD_CCG_NM', 'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD'], axis=1, inplace=True)

# 업종 카운트 데이터 생성

In [191]:
# 업종 카운트 데이터
tmp = df.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'year', 'month']).count()
tmp = tmp[['CNT']].rename(columns={'CNT':'std_cnt'})

In [192]:
# 데이터 정제
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'year', 'month']

df = df.groupby(columns).sum().reset_index(drop=False)
df = df.merge(tmp, how='left', on=columns)
# df = df.merge(flcs, how='left', on=columns)
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,FLC,year,month,CSTMR_CNT,AMT,CNT,std_cnt
0,강원,건강보조식품 소매업,1,2019,1,4,311200,4,1
1,강원,건강보조식품 소매업,1,2019,2,7,1517000,8,2
2,강원,건강보조식품 소매업,1,2019,3,16,982750,17,3
3,강원,건강보조식품 소매업,1,2019,4,4,266000,4,1
4,강원,건강보조식품 소매업,1,2019,5,13,1057200,15,3


In [193]:
# 단골 지수
def regular(x, y):
    if x == 0:
        x += 1
    return y / x

df['regular_power'] = df.apply(lambda x: regular(x['CSTMR_CNT'], x['CNT']), axis=1)

In [194]:
def make_season(data):
    if data in [12, 1, 2]:
        return 'Winter'
    elif data in [3, 4, 5]:
        return 'Spring'
    elif data in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

In [195]:
df['Season'] = df['month'].apply(lambda x: make_season(x))

In [196]:
df['CV'] = 0
df.loc[(df['year']==2020) & (df['month']==1), 'CV'] = 1
df.loc[(df['year']==2020) & (df['month']==2), 'CV'] = 2
df.loc[(df['year']==2020) & (df['month']==3), 'CV'] = 3

In [197]:
df['FLC'] = df['FLC'].astype('object')
df['month'] = df['month'].astype('object')

In [198]:
df

,CARD_SIDO_NM,STD_CLSS_NM,FLC,year,month,CSTMR_CNT,AMT,CNT,std_cnt,regular_power,Season,CV
0,강원,건강보조식품 소매업,1,2019,1,4,311200,4,1,1.000000,Winter,0
1,강원,건강보조식품 소매업,1,2019,2,7,1517000,8,2,1.142857,Winter,0
2,강원,건강보조식품 소매업,1,2019,3,16,982750,17,3,1.062500,Spring,0
3,강원,건강보조식품 소매업,1,2019,4,4,266000,4,1,1.000000,Spring,0
4,강원,건강보조식품 소매업,1,2019,5,13,1057200,15,3,1.153846,Spring,0
...,...,...,...,...,...,...,...,...,...,...,...,...
47858,충북,휴양콘도 운영업,5,2019,12,128,9446100,190,21,1.484375,Winter,0
47859,충북,휴양콘도 운영업,5,2020,1,149,12968610,233,21,1.563758,Winter,1
47860,충북,휴양콘도 운영업,5,2020,2,61,3895100,97,7,1.590164,Winter,2
47861,충북,휴양콘도 운영업,5,2020,3,15,1347800,19,5,1.266667,Spring,3


In [199]:
# 1,2,3,4월 데이터만 사용
df = df[(df['year']==2020) & (df['month'].isin([1,2,3,4]))]

In [210]:
df.reset_index(inplace=True)

In [212]:
df.drop('index', inplace=True, axis=1)

In [213]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [214]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11664 entries, 0 to 11663
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CARD_SIDO_NM   11664 non-null  int32  
 1   STD_CLSS_NM    11664 non-null  int32  
 2   FLC            11664 non-null  int32  
 3   year           11664 non-null  int64  
 4   month          11664 non-null  int32  
 5   CSTMR_CNT      11664 non-null  int64  
 6   AMT            11664 non-null  int64  
 7   CNT            11664 non-null  int64  
 8   std_cnt        11664 non-null  int64  
 9   regular_power  11664 non-null  float64
 10  Season         11664 non-null  int32  
 11  CV             11664 non-null  int64  
dtypes: float64(1), int32(5), int64(6)
memory usage: 865.8 KB


In [215]:
df.CARD_SIDO_NM.unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [219]:
# feature, target 설정
x = df_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(df_num['AMT'])

In [220]:
s_train_ind = s_train_ind = df[~((df['year']==2020) & (df['month']==3)) & (df['CARD_SIDO_NM']=='서울')].index

In [221]:
# 서울, 경기, 그외 지역으로 분리
s_train_ind = df[~((df['year']==2020) & (df['month']==4)) & (df['CARD_SIDO_NM']=='서울')].index
s_valid_ind = df[(df['year']==2020) & (df['month']==4) & (df['CARD_SIDO_NM']=='서울')].index

k_train_ind = df[~((df['year']==2020) & (df['month']==4)) & (df['CARD_SIDO_NM']=='경기')].index
k_valid_ind = df[(df['year']==2020) & (df['month']==4) & (df['CARD_SIDO_NM']=='경기')].index

j_train_ind = df[~((df['year']==2020) & (df['month']==4)) & (df['CARD_SIDO_NM']=='제주')].index
j_valid_ind = df[(df['year']==2020) & (df['month']==4) & (df['CARD_SIDO_NM']=='제주')].index

other_train_ind = df[~((df['year'] == 2020) & (df['month'] == 4)) & ~(df['CARD_SIDO_NM'].isin(['서울', '경기', '제주']))].index
other_valid_ind = df[~df['CARD_SIDO_NM'].isin(['서울', '경기', '제주']) & (df['year'] == 2020) & (df['month'] == 4)].index

In [222]:
x_s_train = x.iloc[s_train_ind]
x_s_val = x.iloc[s_valid_ind]
y_s_train = y.iloc[s_train_ind]
y_s_val = y.iloc[s_valid_ind]

x_k_train = x.iloc[k_train_ind]
x_k_val = x.iloc[k_valid_ind]
y_k_train = y.iloc[k_train_ind]
y_k_val = y.iloc[k_valid_ind]

x_j_train = x.iloc[j_train_ind]
x_j_val = x.iloc[j_valid_ind]
y_j_train = y.iloc[j_train_ind]
y_j_val = y.iloc[j_valid_ind]

x_oth_train = x.iloc[other_train_ind]
x_oth_val = x.iloc[other_valid_ind]
y_oth_train = y.iloc[other_train_ind]
y_oth_val = y.iloc[other_valid_ind]

In [223]:
print(len(x_s_train))
print(len(y_s_train))
print(len(x_s_val))
print(len(y_s_val))

579
579
189
189


In [224]:
XGB_model = XGBRegressor(learning_rate=0.3, max_depth=6, n_estimators=500)
XGB_model.fit(x_s_train, y_s_train)
y_XGB_predict = XGB_model.predict(x_s_val)

In [225]:
mean_squared_error(y_XGB_predict, y_s_val)

0.13813266673599758

  ## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [226]:
import lightgbm as lgb

In [227]:
def rmsle_lgbm(y_pred, data):
    y_true = np.array(data.get_label())
    score = np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))
  
    return 'rmsle', score, False

In [228]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
          # 'lambda_l1' : 0.1,
            'lambda_l2' : 0.1,
          # 'max_depth' : -1,
          # 'bagging_freq' : 5,
          # 'max_bin': 128
        }

In [229]:
# 서울
categorical_features = list(df.dtypes[df.dtypes=='object'].index)
train_s = lgb.Dataset(x_s_train, label=y_s_train, categorical_feature=categorical_features)
val_s = lgb.Dataset(x_s_val, label=y_s_val)

In [230]:
model_s = lgb.train(params,
                  train_s,
                  20000,
                  val_s,
                  verbose_eval = 100,
                  early_stopping_rounds = 100,
                  feval=rmsle_lgbm
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.477405	valid_0's rmsle: 0.0282941
[200]	valid_0's rmse: 0.452958	valid_0's rmsle: 0.0271651
[300]	valid_0's rmse: 0.434414	valid_0's rmsle: 0.026293
[400]	valid_0's rmse: 0.429209	valid_0's rmsle: 0.0260804
[500]	valid_0's rmse: 0.425096	valid_0's rmsle: 0.0258559
[600]	valid_0's rmse: 0.423833	valid_0's rmsle: 0.0257961
[700]	valid_0's rmse: 0.422865	valid_0's rmsle: 0.0257209
[800]	valid_0's rmse: 0.423469	valid_0's rmsle: 0.0257454
Early stopping, best iteration is:
[759]	valid_0's rmse: 0.421916	valid_0's rmsle: 0.0256587


In [231]:
# 경기
categorical_features = list(df.dtypes[df.dtypes=='object'].index)
train_k = lgb.Dataset(x_k_train, label=y_k_train, categorical_feature=categorical_features)
val_k = lgb.Dataset(x_k_val, label=y_k_val)

In [232]:
model_k = lgb.train(params,
                  train_k,
                  20000,
                  val_k,
                  verbose_eval = 100,
                  early_stopping_rounds = 100,
                  feval=rmsle_lgbm
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.364991	valid_0's rmsle: 0.0210897
[200]	valid_0's rmse: 0.315141	valid_0's rmsle: 0.017871
[300]	valid_0's rmse: 0.304194	valid_0's rmsle: 0.017283
[400]	valid_0's rmse: 0.29937	valid_0's rmsle: 0.017186
Early stopping, best iteration is:
[361]	valid_0's rmse: 0.299995	valid_0's rmsle: 0.0171326


In [233]:
# 제주
categorical_features = list(df.dtypes[df.dtypes=='object'].index)
train_j = lgb.Dataset(x_j_train, label=y_j_train, categorical_feature=categorical_features)
val_j = lgb.Dataset(x_j_val, label=y_j_val)

In [234]:
model_j = lgb.train(params,
                  train_j,
                  20000,
                  val_j,
                  verbose_eval = 100,
                  early_stopping_rounds = 100,
                  feval=rmsle_lgbm
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.427564	valid_0's rmsle: 0.0253358
[200]	valid_0's rmse: 0.399076	valid_0's rmsle: 0.0240303
[300]	valid_0's rmse: 0.398331	valid_0's rmsle: 0.0240701
Early stopping, best iteration is:
[231]	valid_0's rmse: 0.397524	valid_0's rmsle: 0.0239718


In [235]:
# 그 외 지역
categorical_features = list(df.dtypes[df.dtypes=='object'].index)
train_oth = lgb.Dataset(x_oth_train, label=y_oth_train, categorical_feature=categorical_features)
val_oth = lgb.Dataset(x_oth_val, label=y_oth_val)

In [236]:
model_oth = lgb.train(params,
                  train_oth,
                  20000,
                  val_oth,
                  verbose_eval = 100,
                  early_stopping_rounds = 100,
                  feval=rmsle_lgbm
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.385958	valid_0's rmsle: 0.0244044
[200]	valid_0's rmse: 0.345464	valid_0's rmsle: 0.0222508
[300]	valid_0's rmse: 0.332495	valid_0's rmsle: 0.0216691
[400]	valid_0's rmse: 0.326753	valid_0's rmsle: 0.0214365
[500]	valid_0's rmse: 0.323551	valid_0's rmsle: 0.0213132
[600]	valid_0's rmse: 0.319349	valid_0's rmsle: 0.0211352
[700]	valid_0's rmse: 0.316462	valid_0's rmsle: 0.0210253
[800]	valid_0's rmse: 0.313785	valid_0's rmsle: 0.0209144
[900]	valid_0's rmse: 0.312828	valid_0's rmsle: 0.0208889
[1000]	valid_0's rmse: 0.311596	valid_0's rmsle: 0.020849
[1100]	valid_0's rmse: 0.310605	valid_0's rmsle: 0.0208139
[1200]	valid_0's rmse: 0.31011	valid_0's rmsle: 0.0207978
[1300]	valid_0's rmse: 0.310109	valid_0's rmsle: 0.0208168
Early stopping, best iteration is:
[1230]	valid_0's rmse: 0.309859	valid_0's rmsle: 0.0207881


## 6. 결과 및 결언
## Conclusion & Discussion

In [237]:
df_num.head()

,CARD_SIDO_NM,STD_CLSS_NM,FLC,year,month,CSTMR_CNT,AMT,CNT,std_cnt,regular_power,Season,CV
0,0,0,0,2020,0,19,933146,19,5,1.000000,1,1
1,0,0,0,2020,1,9,441100,8,2,0.888889,1,2
2,0,0,0,2020,2,3,427510,2,1,0.666667,0,3
3,0,0,0,2020,3,6,478500,5,2,0.833333,0,0
4,0,0,1,2020,0,160,21885035,162,17,1.012500,1,1


In [238]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
FLCs          = df_num['FLC'].unique()
CVs           = [3]
Seasons       = df_num['Season'].unique()
years         = [2020]
months        = [3, 6]

temp = []
from itertools import product
comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs, FLCs, years, months, CVs]
temp = np.array(list(product(*comb_list)))
temp = pd.DataFrame(data=temp, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'year', 'month', 'CV'])

In [239]:
df['Season'].unique()

array(['Winter', 'Spring'], dtype=object)

In [240]:
def m_season(x):
    if x==3:
        return 1
    if x==6:
        return 2

In [241]:
temp['Season'] = temp['month'].apply(lambda x: m_season(x))

In [242]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,FLC,year,month,CV,Season
0,0,0,0,2020,3,3,1
1,0,0,0,2020,6,3,2
2,0,0,1,2020,3,3,1
3,0,0,1,2020,6,3,2
4,0,0,2,2020,3,3,1
...,...,...,...,...,...,...,...
6965,16,30,2,2020,6,3,2
6966,16,30,3,2020,3,3,1
6967,16,30,3,2020,6,3,2
6968,16,30,4,2020,3,3,1


In [246]:
# 템플렛에 넣을 4월 데이터 가져오기

tmp = df_num[(df['year']==2020) & (df['month']==4)]
tmp = tmp.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'month'])[['CSTMR_CNT', 'CNT', 'std_cnt']].sum().reset_index(drop=False)
tmp['regular_power'] = tmp.apply(lambda x : math.ceil(regular(x['CSTMR_CNT'], x['CNT'])/2), axis=1)
tmp.drop(['CSTMR_CNT', 'CNT'], axis=1, inplace=True)
april = tmp
april

,CARD_SIDO_NM,STD_CLSS_NM,FLC,month,std_cnt,regular_power
0,0,0,0,3,2,1
1,0,0,1,3,11,1
2,0,0,2,3,8,1
3,0,0,3,3,22,1
4,0,0,4,3,27,1
...,...,...,...,...,...,...
2876,16,40,0,3,3,1
2877,16,40,1,3,11,1
2878,16,40,2,3,8,1
2879,16,40,3,3,8,1


In [247]:
# 템플렛에 넣을 7월 데이터 가져오기

tmp = df_num[(df_num['year']==2020) & (df['month']==4)]
tmp = tmp.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'month'])[['CSTMR_CNT', 'CNT', 'std_cnt']].sum().reset_index(drop=False)
tmp['regular_power'] = tmp.apply(lambda x : math.ceil(regular(x['CSTMR_CNT'], x['CNT'])/2), axis=1)
tmp.drop(['CSTMR_CNT', 'CNT'], axis=1, inplace=True)
july = tmp
july

,CARD_SIDO_NM,STD_CLSS_NM,FLC,month,std_cnt,regular_power
0,0,0,0,3,2,1
1,0,0,1,3,11,1
2,0,0,2,3,8,1
3,0,0,3,3,22,1
4,0,0,4,3,27,1
...,...,...,...,...,...,...
2876,16,40,0,3,3,1
2877,16,40,1,3,11,1
2878,16,40,2,3,8,1
2879,16,40,3,3,8,1


In [248]:
tmp = pd.concat([april, july], axis=0)

In [249]:
# # 7월은 // 2하면 안됨. 수정 필요
# import math

# tmp = df_num.groupby(['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'month'])[['CSTMR_CNT', 'CNT', 'std_cnt']].sum().reset_index(drop=False)
# tmp['regular_power'] = tmp.apply(lambda x : math.ceil(regular(x['CSTMR_CNT'], x['CNT'])/2), axis=1)
# tmp['std_cnt'] = tmp['std_cnt'] // 2 + 1
# tmp = tmp[tmp['month'].isin(months)].reset_index(drop=True)
# tmp.drop(['CSTMR_CNT', 'CNT', 'regular_power'], axis=1, inplace=True)
# tmp

In [250]:
temp = temp.merge(tmp, on=['CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'month'], how='left')
temp.fillna(1.0, inplace=True)

In [251]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,FLC,year,month,CV,Season,std_cnt,regular_power
0,0,0,0,2020,3,3,1,2.0,1.0
1,0,0,0,2020,3,3,1,2.0,1.0
2,0,0,0,2020,6,3,2,1.0,1.0
3,0,0,1,2020,3,3,1,11.0,1.0
4,0,0,1,2020,3,3,1,11.0,1.0
...,...,...,...,...,...,...,...,...,...
9846,16,30,2,2020,6,3,2,1.0,1.0
9847,16,30,3,2020,3,3,1,1.0,1.0
9848,16,30,3,2020,6,3,2,1.0,1.0
9849,16,30,4,2020,3,3,1,1.0,1.0


In [252]:
df.CARD_SIDO_NM.unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [253]:
# 예측
pred_s = model_s.predict(temp[temp['CARD_SIDO_NM'] == 8])
pred_s = np.expm1(pred_s)

pred_k = model_k.predict(temp[temp['CARD_SIDO_NM'] == 1])
pred_k = np.expm1(pred_k)

pred_j = model_j.predict(temp[temp['CARD_SIDO_NM'] == 14])
pred_j = np.expm1(pred_j)

pred_oth = model_oth.predict(temp[~temp['CARD_SIDO_NM'].isin([1,8,14])])
pred_oth = np.expm1(pred_oth)

In [254]:
# 디코딩
temp.loc[temp['CARD_SIDO_NM'] == 8, 'AMT'] = np.round(pred_s, 0)
temp.loc[temp['CARD_SIDO_NM'] == 1, 'AMT'] = np.round(pred_k, 0)
temp.loc[temp['CARD_SIDO_NM'] == 14, 'AMT'] = np.round(pred_j, 0)
temp.loc[~temp['CARD_SIDO_NM'].isin([1,8,14]), 'AMT'] = np.round(pred_oth, 0)

temp['REG_YYMM'] = temp['year']*100 + temp['month'] + 1
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index(drop=False)
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [255]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,10753212.0
1,202004,강원,골프장 운영업,9547208.0
2,202004,강원,과실 및 채소 소매업,8620590.0
3,202004,강원,관광 민예품 및 선물용품 소매업,2896352.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1886506.0
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,8195258.0
1390,202007,충북,한식 음식점업,7471372.0
1391,202007,충북,호텔업,3005972.0
1392,202007,충북,화장품 및 방향제 소매업,6695741.0


In [62]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,26955061.0
1,202004,강원,골프장 운영업,51543800.0
2,202004,강원,과실 및 채소 소매업,30242750.0
3,202004,강원,관광 민예품 및 선물용품 소매업,6210941.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,4892340.0
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,19470469.0
1390,202007,충북,한식 음식점업,19677347.0
1391,202007,충북,호텔업,6752975.0
1392,202007,충북,화장품 및 방향제 소매업,22244880.0


In [256]:
# 2020년 3월 기준 특정 업종이 없는 지역
sido = df['CARD_SIDO_NM'].unique()
std = df['STD_CLSS_NM'].unique()

tmp = df[(df['month'] == 4) & (df['year'] == 2020)]
tmp = tmp[['CARD_SIDO_NM', 'STD_CLSS_NM']]
tmp = tmp.drop_duplicates(['CARD_SIDO_NM', 'STD_CLSS_NM'], keep='first').reset_index(drop=True)

d = {}

for s in sido:
    d[s] = []
    for c in std:
        d[s].append(c)

for i in tmp.index:
    d[tmp.loc[i, 'CARD_SIDO_NM']].remove(tmp.loc[i, 'STD_CLSS_NM'])

In [257]:
for i in temp.index:
    for k, v in d.items():
        if k == temp.loc[i, 'CARD_SIDO_NM']:
            if temp.loc[i, 'STD_CLSS_NM'] in v:
                temp.loc[i, 'AMT'] = 0

In [258]:
# 제출 파일 만들기
submission = pd.read_csv('./data/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'

In [259]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,10753212.0
1,202004,강원,골프장 운영업,9547208.0
2,202004,강원,과실 및 채소 소매업,8620590.0
3,202004,강원,관광 민예품 및 선물용품 소매업,2896352.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.0
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,8195258.0
1390,202007,충북,한식 음식점업,7471372.0
1391,202007,충북,호텔업,3005972.0


In [260]:
submission.to_csv('submission_0730.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,10753212.0
1,202004,강원,골프장 운영업,9547208.0
2,202004,강원,과실 및 채소 소매업,8620590.0
3,202004,강원,관광 민예품 및 선물용품 소매업,2896352.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.0


In [261]:
submission.tail()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,8195258.0
1390,202007,충북,한식 음식점업,7471372.0
1391,202007,충북,호텔업,3005972.0
1392,202007,충북,화장품 및 방향제 소매업,6695741.0
1393,202007,충북,휴양콘도 운영업,4714666.0


In [262]:
tmp22 = df_raw.loc[(df_raw['REG_YYMM']==202004)]
tmp23 = tmp22.groupby(['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index()

In [263]:
tmp24 = pd.merge(submission, tmp23, on=['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM'])
tmp24['AMT_diff'] = ((tmp24['AMT_y'] - tmp24['AMT_x']) / tmp24['AMT_y']) * 100

In [264]:
tmp24

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_x,AMT_y,AMT_diff
0,202004,강원,건강보조식품 소매업,10753212.0,88823988,87.893797
1,202004,강원,골프장 운영업,9547208.0,4708346820,99.797228
2,202004,강원,과실 및 채소 소매업,8620590.0,1121028924,99.231011
3,202004,강원,관광 민예품 및 선물용품 소매업,2896352.0,14360780,79.831513
4,202004,강원,그외 기타 스포츠시설 운영업,5046470.0,227200,-2121.157570
...,...,...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,11709082.0,1373635928,99.147585
606,202004,충북,한식 음식점업,8909312.0,18911036160,99.952888
607,202004,충북,호텔업,3045290.0,14121500,78.435081
608,202004,충북,화장품 및 방향제 소매업,7939294.0,450507431,98.237700


In [265]:
np.sqrt(mean_squared_error(tmp24['AMT_y'],tmp24['AMT_x']))

17769165349.742058